# Amélioration des coordonnées & adresses de l'annuaire des écoles
**Objectifs :**
- Améliorer la qualité des coordonnées des établissements scolaires (champs geom de la table etablissements)
- Bonus : meilleure complétude des champs liés à l'adresse des établissements scolaires (notamment le champs adresse_3 de la table etablissements)

**Livrables :** 
- Estimer le nb / % d'établissements scolaires pour lesquels coordonnées & adresses semblent incohérents
- Script python avec la logique permettant l'amélioration de la qualité des données de l'annuaire
- Optionnel : migration alembic pour mettre à jour la base de données potentiel_solaire.duckdb (@machbry s'en occupera si besoin)

## Python stuff

In [1]:
import requests
import pandas as pd
import geopandas as gpd
from typing import Optional
import plotly.express as px
from shapely.geometry import Point
from potentiel_solaire.constants import DATA_FOLDER
from potentiel_solaire.database.queries import get_connection

## 1 - Chargement des données et exploration des données
> Code de l'import des données est fourni dans l'[issue 245](https://github.com/dataforgoodfr/13_potentiel_solaire/issues/245).

### 1.1 - Chargement des données

In [2]:
# Export des etablissements au format des .geojson
output_path = f"{DATA_FOLDER}/etablissements.geojson"

with get_connection() as conn:
    export_query = f"""
        COPY (
        SELECT *
        FROM
            etablissements
        ) TO '{output_path}' WITH (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_NAME 'Etablissements')
    """

    conn.query(export_query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
# Lecture du fichier GeoJSON
use_cols = [
    "identifiant_de_l_etablissement", 
    "nom_etablissement", 
    "code_commune", 
    "adresse_1", 
    "adresse_2", 
    "adresse_3", 
    "code_postal", 
    "identifiant_topo_zone_rattachee", 
    "geometry"
]
etablissements = gpd.read_file(output_path)
f_etablissements = etablissements[use_cols]

### 1.2 - Exploration rapide des données

In [4]:
print(f"--> {f_etablissements.shape[0]} établissements scolaires chargés.")
print(f"--> Projection spatiale : {f_etablissements.crs}")
f_etablissements.head()

--> 52546 établissements scolaires chargés.
--> Projection spatiale : EPSG:4326


,identifiant_de_l_etablissement,nom_etablissement,code_commune,adresse_1,adresse_2,adresse_3,code_postal,identifiant_topo_zone_rattachee,geometry
0,0631701G,Ecole maternelle Jean Alix,63284,Chemin DU MOULIN,None,63430 PONT DU CHATEAU,63430,None,POINT (3.25496 45.79291)
1,0631702H,Ecole maternelle Montjoie,63471,7 rue de Montjoie,None,63700 YOUX,63700,None,POINT (2.81888 46.1339)
2,0631743C,Ecole élémentaire René Cassin,63300,Avenue du Général de Gaulle,None,63200 RIOM,63200,None,POINT (3.10421 45.88063)
3,0631756S,Ecole maternelle George Sand,63113,Rue de Neyrat,None,63100 CLERMONT FERRAND,63100,None,POINT (3.10179 45.81044)
4,0631757T,Ecole élémentaire George Sand,63113,71 rue de Neyrat,None,63000 CLERMONT FERRAND,63000,None,POINT (3.10282 45.80544)


In [5]:
f_etablissements.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 52546 entries, 0 to 52545
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   identifiant_de_l_etablissement   52546 non-null  object  
 1   nom_etablissement                52546 non-null  object  
 2   code_commune                     52546 non-null  object  
 3   adresse_1                        52531 non-null  object  
 4   adresse_2                        10365 non-null  object  
 5   adresse_3                        44558 non-null  object  
 6   code_postal                      52546 non-null  object  
 7   identifiant_topo_zone_rattachee  0 non-null      object  
 8   geometry                         52539 non-null  geometry
dtypes: geometry(1), object(8)
memory usage: 3.6+ MB


### 1.3 - Les adresses/geometrie vide

In [6]:
# Géometrie null --> piste de remplissage geocodage de la BAN
f_etablissements[f_etablissements["geometry"].isnull()]

,identifiant_de_l_etablissement,nom_etablissement,code_commune,adresse_1,adresse_2,adresse_3,code_postal,identifiant_topo_zone_rattachee,geometry
30173,9730606W,SEGPA CLG REMIRE-MONTJOLY III,97309,"7 avenue L.G. DAMAS, Eco Quartier Geor",None,97354 REMIRE MONTJOLY,97354,None,None
30809,0741834S,Section d'enseignement générale et professionn...,74229,525 chemin de la fondue,None,74140 ST CERGUES,74140,None,None
31144,0342605X,Lycée Polyvalent de Cournonterral,34088,2 avenue Cynisca,None,34660 COURNONTERRAL,34660,None,None
31145,0342608A,Section d'enseignement professionnel du Lycée ...,34088,2 avenue Cynisca,None,34660 COURNONTERRAL,34660,None,None
31811,9741966U,Ecole maternelle publique de Beauséjour Sainte...,97418,Quartier de Beauséjour,None,97438 STE MARIE,97438,None,None
45976,0741835T,Collège de la Géline - Vetraz-Monthoux,74298,Chemin des Fontaines,None,None,74100,None,None
51598,0741833R,Collège de Saint-Cergues,74229,Route du Bourgeau,None,None,74140,None,None


> Proposition : remplir les données vie geocodage par la BAN

In [7]:
f_etablissements[f_etablissements["adresse_1"].isnull()].head()

,identifiant_de_l_etablissement,nom_etablissement,code_commune,adresse_1,adresse_2,adresse_3,code_postal,identifiant_topo_zone_rattachee,geometry
11600,9760206V,Lycée agricole de Coconi,97614,None,BP 02,None,97670,None,POINT (45.13416 -12.83958)
13466,0291758D,Centre d'instruction naval Ecole de Maistrance,29019,None,BP 300,29240 BREST CEDEX 9,29240,None,POINT (-4.48606 48.39043)
27398,9760543L,Ecole élémentaire publique de Iloni T12,97607,None,None,97660 DEMBENI,97660,None,POINT (45.18777 -12.84673)
27966,9760244L,Collège Frédéric d'Achery,97610,None,None,None,97690,None,POINT (45.20251 -12.73505)
28012,9760125G,Lycée polyvalent Gustave Eiffel de Kahani,97614,None,BP 04,None,97670,None,POINT (45.13107 -12.82231)


In [8]:
f_etablissements = f_etablissements.copy()
f_etablissements["adresse"] = (
    f_etablissements["adresse_2"].fillna('') + ' '
    + f_etablissements["adresse_1"].fillna('') + ' '
    + f_etablissements["adresse_3"].fillna('')
)
f_etablissements.head()

,identifiant_de_l_etablissement,nom_etablissement,code_commune,adresse_1,adresse_2,adresse_3,code_postal,identifiant_topo_zone_rattachee,geometry,adresse
0,0631701G,Ecole maternelle Jean Alix,63284,Chemin DU MOULIN,None,63430 PONT DU CHATEAU,63430,None,POINT (3.25496 45.79291),Chemin DU MOULIN 63430 PONT DU CHATEAU
1,0631702H,Ecole maternelle Montjoie,63471,7 rue de Montjoie,None,63700 YOUX,63700,None,POINT (2.81888 46.1339),7 rue de Montjoie 63700 YOUX
2,0631743C,Ecole élémentaire René Cassin,63300,Avenue du Général de Gaulle,None,63200 RIOM,63200,None,POINT (3.10421 45.88063),Avenue du Général de Gaulle 63200 RIOM
3,0631756S,Ecole maternelle George Sand,63113,Rue de Neyrat,None,63100 CLERMONT FERRAND,63100,None,POINT (3.10179 45.81044),Rue de Neyrat 63100 CLERMONT FERRAND
4,0631757T,Ecole élémentaire George Sand,63113,71 rue de Neyrat,None,63000 CLERMONT FERRAND,63000,None,POINT (3.10282 45.80544),71 rue de Neyrat 63000 CLERMONT FERRAND


In [9]:
f_etablissements["adresse"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 52546 entries, 0 to 52545
Series name: adresse
Non-Null Count  Dtype 
--------------  ----- 
52546 non-null  object
dtypes: object(1)
memory usage: 410.6+ KB


> Aucune adresse n'est vide

## 2 - Exploration du cas spécifique de l'issue #245

### 2.1 - Rappel de l'issue
**Problème :** La position est incohérente avec les autres données comme adresse, google maps, zone d'education associée, ...

> Par exemple : l'établissement Lycée polyvalent Germaine Tillion (identifiant_de_l_etablissement = 0110012D) a une geom incohérente avec son adresse Avenue du Docteur Laennec BP 1301.

Dans ce cas, la `geom` semble mauvaise car :
- cela place l'établissement proche de l'avenue Georges Pompidou / Rue Jean Durand
- sa zone d'éducation associée est bien située le long de l'avenue du Docteur Laennec

In [10]:
# Charger la données
f_etablissements[f_etablissements["identifiant_de_l_etablissement"] == "0110012D"]

,identifiant_de_l_etablissement,nom_etablissement,code_commune,adresse_1,adresse_2,adresse_3,code_postal,identifiant_topo_zone_rattachee,geometry,adresse
51894,0110012D,Lycée polyvalent Germaine Tillion,11076,Avenue du Docteur Laennec,BP 1301,None,11493,None,POINT (1.94887 43.31734),BP 1301 Avenue du Docteur Laennec


> Note : l'adresse a l'air  bonne mais la géolocalisation n'est pas bonne; exploration d'une première piste avec le geocodeur de la BAN

### 2.2 - API Geocodeur de la BAN

#### 2.2.1 - Script de geocodage

In [11]:
# Fonction de geocodage par la BAN
def geocode_ban(adresse, limit=1):
    """
    Interroge l’API de la BAN pour géocoder une adresse.
    Renvoie la liste des top `limit` résultats.
    """
    url = "https://api-adresse.data.gouv.fr/search/"
    params = {
        "q": adresse, 
        "limit": limit,
        "type": "street"
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    return data["features"][0]["geometry"]["coordinates"], data["features"][0]["properties"]

#### 2.2.2 - Test sur l'adresse dans le GeoDataFrame

In [12]:
# Définir les éléments à géocoder
t_adresse = str(f_etablissements[f_etablissements["identifiant_de_l_etablissement"] == "0110012D"]["adresse"].values[0])
code_postal = str(f_etablissements[f_etablissements["identifiant_de_l_etablissement"] == "0110012D"]["code_postal"].values[0])
adresse = t_adresse + " " + code_postal
results_geom, results_properties = geocode_ban(adresse)

In [13]:
print(f"Adresse recherchée --> {t_adresse}{code_postal}\n")
print("--- RESULTATS DU GEOCODAGE BAN ---")
for key, value in results_properties.items():
    print(f"- {key}: {value}")
print("--- --- ---\n")
print(f"GeoPoint WGG:84 : {results_geom}")

Adresse recherchée --> BP 1301 Avenue du Docteur Laennec 11493

--- RESULTATS DU GEOCODAGE BAN ---
- label: Avenue du Docteur Laennec 44800 Saint-Herblain
- score: 0.5789445933014353
- id: 44162_0890
- banId: cba9b45c-c3de-4bad-9ca1-a52add54ca92
- name: Avenue du Docteur Laennec
- postcode: 44800
- citycode: 44162
- x: 352761.96
- y: 6691947.32
- city: Saint-Herblain
- context: 44, Loire-Atlantique, Pays de la Loire
- type: street
- importance: 0.68418
- street: Avenue du Docteur Laennec
- _type: address
--- --- ---

GeoPoint WGG:84 : [-1.592453, 47.236923]


#### 2.2.3 - Test sur l'adresse ajustée

In [14]:
# Définir les éléments à géocoder
t_adresse = str(f_etablissements[f_etablissements["identifiant_de_l_etablissement"] == "0110012D"]["adresse"].values[0])
code_postal = "11400"
adresse = t_adresse + " " + code_postal
results_geom, results_properties = geocode_ban(adresse)

In [15]:
print(f"Adresse recherchée --> {t_adresse}11400\n")
print("--- RESULTATS DU GEOCODAGE BAN ---")
for key, value in results_properties.items():
    print(f"- {key}: {value}")
print("--- --- ---\n")
print(f"GeoPoint WGG:84 : {results_geom}")

Adresse recherchée --> BP 1301 Avenue du Docteur Laennec 11400

--- RESULTATS DU GEOCODAGE BAN ---
- label: Avenue du Docteur René Laënnec 11400 Castelnaudary
- score: 0.6243590697674418
- id: 11076_0899
- banId: 11d56225-f4d4-472d-9a8f-bcc8050ae73c
- name: Avenue du Docteur René Laënnec
- postcode: 11400
- citycode: 11076
- x: 613960.47
- y: 6245865.38
- city: Castelnaudary
- context: 11, Aude, Occitanie
- type: street
- importance: 0.58888
- street: Avenue du Docteur René Laënnec
- _type: address
--- --- ---

GeoPoint WGG:84 : [1.940132, 43.306754]


> L'adresse : BP 1301 Avenue du Docteur Laennec 11493

On est ici face à un CEDEX  ce qui fait que le géocodage de cette adresse via la BAN ou autre outil sera faux par défaut comme l'exemple ci-dessus le montre. 

L'adresse en elle même n'est pas bonne : sur leurs site internet, on a : 
- LYCEE GERMAINE TILLION 1 avenue du Campus Jean Durand 11493 CASTELNAUDARY CEDEX
> [source](https://germaine-tillion.mon-ent-occitanie.fr/)

Avec le changement du CEDEX avec le vrai code postal, le Géocodage est plus efficace et réussi à modifier légèrement l'emplacement

In [16]:
v_etablissements = f_etablissements[f_etablissements["identifiant_de_l_etablissement"] == "0110012D"].copy()
v_etablissements["geometry"] = Point(results_geom)

In [17]:
# Sélection des données
point = v_etablissements[v_etablissements["identifiant_de_l_etablissement"] == "0110012D"].copy()

# Extraire lat et lon
point["lon"] = point.geometry.x
point["lat"] = point.geometry.y
lon0 = point["lon"].iloc[0]
lat0 = point["lat"].iloc[0]

# Tracer les données
fig = px.scatter_map(
    point,
    lon="lon",
    lat="lat",
    hover_name="identifiant_de_l_etablissement",
    color_discrete_sequence=["gold"],
    size=[15],
    center={"lat": lat0, "lon": lon0},
    zoom=15, 
    height=400, width=400
)
fig.update_layout(
    mapbox_style="open-street-map", 
    margin={"l":0,"r":0,"t":0,"b":0}
)
fig.show()

## 3 - Livrables issue #245

### 3.1 - Identification des données incohérentes
<!-- > Méthodologie : mapper les données avec les données de correspondance entre les codes postaux et codes insee des communes francaises données sur [data.gouv.fr](https://www.data.gouv.fr/datasets/correspondance-entre-les-codes-postaux-et-codes-insee-des-communes-francaises/). 
Les données sont sur opendatasoft, téléchargement du fichier en FlatGeoBuf -->

> Méthodologie : lancer un géocodage avec la BAN correspondance code postaux voir si pas possible 

In [18]:
f_etablissements

,identifiant_de_l_etablissement,nom_etablissement,code_commune,adresse_1,adresse_2,adresse_3,code_postal,identifiant_topo_zone_rattachee,geometry,adresse
0,0631701G,Ecole maternelle Jean Alix,63284,Chemin DU MOULIN,None,63430 PONT DU CHATEAU,63430,None,POINT (3.25496 45.79291),Chemin DU MOULIN 63430 PONT DU CHATEAU
1,0631702H,Ecole maternelle Montjoie,63471,7 rue de Montjoie,None,63700 YOUX,63700,None,POINT (2.81888 46.1339),7 rue de Montjoie 63700 YOUX
2,0631743C,Ecole élémentaire René Cassin,63300,Avenue du Général de Gaulle,None,63200 RIOM,63200,None,POINT (3.10421 45.88063),Avenue du Général de Gaulle 63200 RIOM
3,0631756S,Ecole maternelle George Sand,63113,Rue de Neyrat,None,63100 CLERMONT FERRAND,63100,None,POINT (3.10179 45.81044),Rue de Neyrat 63100 CLERMONT FERRAND
4,0631757T,Ecole élémentaire George Sand,63113,71 rue de Neyrat,None,63000 CLERMONT FERRAND,63000,None,POINT (3.10282 45.80544),71 rue de Neyrat 63000 CLERMONT FERRAND
...,...,...,...,...,...,...,...,...,...,...
52541,9741354D,Section d'enseignement général et technologiqu...,97411,CITE SCOLAIRE DU BUTOR,BUTOR / BP 155,97492 STE CLOTILDE CEDEX,97492,None,POINT (55.47051 -20.88848),BUTOR / BP 155 CITE SCOLAIRE DU BUTOR 97492 ST...
52542,9741558A,Section d'enseignement professionnel du Lycée ...,97412,10 ROUTE DE LA MARINE,None,97480 ST JOSEPH,97480,None,POINT (55.66822 -21.37716),10 ROUTE DE LA MARINE 97480 ST JOSEPH
52543,9741624X,Section d'enseignement professionnel du Lycée ...,97411,Route francicéas,Bois de Nèfles,97400 ST DENIS,97400,None,POINT (55.47752 -20.91054),Bois de Nèfles Route francicéas 97400 ST DENIS
52544,9760296T,SEP du lycée de Chirongui,97606,CHIRONGUI,CHIRONGUI,97620 CHIRONGUI,97620,None,POINT (45.15056 -12.93488),CHIRONGUI CHIRONGUI 97620 CHIRONGUI


In [19]:
# Geocodage --> tester de géocoder pour améliorer les adresses et détecter les annomalies
f_etablissements["adresse"]

0                   Chemin DU MOULIN 63430 PONT DU CHATEAU
1                             7 rue de Montjoie 63700 YOUX
2                   Avenue du Général de Gaulle 63200 RIOM
3                     Rue de Neyrat 63100 CLERMONT FERRAND
4                  71 rue de Neyrat 63000 CLERMONT FERRAND
                               ...                        
52541    BUTOR / BP 155 CITE SCOLAIRE DU BUTOR 97492 ST...
52542               10  ROUTE DE LA MARINE 97480 ST JOSEPH
52543       Bois de Nèfles Route francicéas 97400 ST DENIS
52544                  CHIRONGUI CHIRONGUI 97620 CHIRONGUI
52545                          KWALE KWALE 97605 MAMOUDZOU
Name: adresse, Length: 52546, dtype: object

In [20]:
f_etablissements["code_postal"]

0        63430
1        63700
2        63200
3        63100
4        63000
         ...  
52541    97492
52542    97480
52543    97400
52544    97620
52545    97605
Name: code_postal, Length: 52546, dtype: object

In [21]:
csv_path = "../data/correspondance-code-insee-code-postal.csv"
gdf_corr = pd.read_csv(csv_path,  sep=";", 
    encoding="latin1",    # <- change l'encodage
    engine="python",      # <- parfois plus tolérant
    dtype=str )
gdf_corr.head()

,#Code_commune_INSEE,Nom_de_la_commune,Code_postal,Libellé_d_acheminement,Ligne_5
0,01001,L ABERGEMENT CLEMENCIAT,01400,L ABERGEMENT CLEMENCIAT,NaN
1,01002,L ABERGEMENT DE VAREY,01640,L ABERGEMENT DE VAREY,NaN
2,01004,AMBERIEU EN BUGEY,01500,AMBERIEU EN BUGEY,NaN
3,01005,AMBERIEUX EN DOMBES,01330,AMBERIEUX EN DOMBES,NaN
4,01006,AMBLEON,01300,AMBLEON,NaN


In [28]:
# 2️⃣ Extraction des colonnes utiles et conversion en DataFrame pandas
df_corr = pd.DataFrame({
    "code_commune": gdf_corr["#Code_commune_INSEE"].astype(str),
    "postal_code_ref": gdf_corr["Code_postal"].astype(str),
})

# 3️⃣ On enlève les doublons (au cas où)
df_corr = df_corr.drop_duplicates(subset=["code_commune", "postal_code_ref"])

# 4️⃣ Merge avec vos établissements
#    On suppose que f_etablissements est déjà un DataFrame pandas
#    avec les colonnes 'code_commune' et 'code_postal'
merged = f_etablissements[f_etablissements["adresse_3"].isnull()].merge(
    df_corr,
    on="code_commune",
    how="left"
)

# 5️⃣ Vérification de la correspondance des codes postaux
merged["postal_ok"] = merged["code_postal"].astype(str) == merged["postal_code_ref"]

# 6️⃣ Résultat final
data_merged = merged[["identifiant_de_l_etablissement", "code_commune", "code_postal", "postal_code_ref", "postal_ok"]]
data_merged[data_merged["postal_ok"] == False].to_csv("../data/etablissements_postal_check.csv", index=False)


In [29]:
data_merged[data_merged["postal_ok"] == False]

,identifiant_de_l_etablissement,code_commune,code_postal,postal_code_ref,postal_ok
1,0313080E,31555,31000,31100,False
2,0313080E,31555,31000,31200,False
3,0313080E,31555,31000,31300,False
4,0313080E,31555,31000,31400,False
5,0313080E,31555,31000,31500,False
...,...,...,...,...,...
10081,0630022G,63113,63100,63000,False
10083,0630018C,63113,63037,63000,False
10084,0630018C,63113,63037,63100,False
10088,0030061A,03095,03306,03300,False


In [23]:
f_etablissements.code_commune.nunique()

20827

In [30]:
f_etablissements[f_etablissements["adresse_3"].isnull()]

,identifiant_de_l_etablissement,nom_etablissement,code_commune,adresse_1,adresse_2,adresse_3,code_postal,identifiant_topo_zone_rattachee,geometry,adresse
8550,0313080E,Collège Jeanne et Jean Philippe,31555,7 rue Micheline Ostermeyer,None,None,31000,None,POINT (1.39302 43.56027),7 rue Micheline Ostermeyer
9550,0313228R,Lycée polyvalent d'Auterive,31033,600 route d'Espagne,None,None,31190,None,POINT (1.47471 43.3448),600 route d'Espagne
9551,9730094P,Lycée professionnel Jean Marie Michotte,97302,Boulevard de la République,BP 5018,None,97305,None,POINT (-52.32394 4.938),BP 5018 Boulevard de la République
9552,0061760F,Lycée Alexis de Tocqueville,06069,22 chemin de l'Orme,BP 72111,None,06131,None,POINT (6.94095 43.65363),BP 72111 22 chemin de l'Orme
9553,0142133T,Lycée Laplace - Dumont d'Urville,14118,130 rue de la Délivrande,BP 75183,None,14075,None,POINT (-0.35722 49.19631),BP 75183 130 rue de la Délivrande
...,...,...,...,...,...,...,...,...,...,...
52240,0031044U,Lycée polyvalent Geneviève Vincent,03082,15 boulevard du Général de Gaulle,None,None,03600,None,POINT (2.73495 46.29235),15 boulevard du Général de Gaulle
52241,0430112N,LEGTA de Brioude Bonnefont Saugues - site de F...,43096,Route de Bonnefont,None,None,43100,None,POINT (3.42341 45.29192),Route de Bonnefont
52242,0150030B,Lycée polyvalent de Haute Auvergne,15187,20 rue Marcellin Boudet,BP 41,None,15101,None,POINT (3.08198 45.0352),BP 41 20 rue Marcellin Boudet
52243,0430023S,Lycée professionnel Auguste Aymard,43089,2 rue Saint Marcel,None,None,43000,None,POINT (3.86845 45.04909),2 rue Saint Marcel


> Conclusion : Mauvaise approche